In [1]:
#selenium 설치
!pip install selenium

from selenium import webdriver
import pandas as pd


In [2]:
driver = webdriver.Chrome("chromedriver.exe") 

In [3]:
idol_list = pd.read_csv('92-93_idol.csv', encoding = 'utf-8')

In [4]:
print(idol_list)

     artist         artist_eng  member_num leader_name gender          agency
0  서태지와 아이들  SeoTaiji And Boys           3         서태지      남           요요 기획
1      더 블루           The Blue           2         손지창      남       SM 엔터테인먼트
2         잼                Zam           5         조진수      혼         DSP 미디어
3     에스오에스              S.O.S           4         NaN      여         삼포니 레코드
4       솔리드              Solid           3         정재윤      남             비앤비
5        듀스               DEUX           2         이현도      남  예당 음향 (예당 컴퍼니)
6       노이즈              Noise           4         홍종구      남           라인 기획


In [17]:
singer = idol_list['artist']

singer_id =[]

In [19]:
for i in singer:
    driver.get("https://www.melon.com/index.htm")   
    search = driver.find_element_by_xpath('//*[@id="top_search"]') #검색창 xpath
    search.clear() #검색창 clear
    search.send_keys(i+'\n')  #검색창에 가수 입력후 엔터

    try:
        driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div/div[1]/div/a/strong').click() #검색한 아티스트 클릭
        url = driver.current_url
        id = url.replace('https://www.melon.com/artist/timeline.htm?artistId=', '') #url에 있는 가수 고유번호 저장    
        singer_id.append(id)
    except:
        singer_id.append('')
  

In [25]:
singer_id

['100161', '180144', '101781', '', '100037', '100977', '100025']

In [26]:
id = pd.DataFrame(singer_id)

In [30]:
id.head()

,0
0,100161
1,180144
2,101781
3,
4,100037


In [31]:
id.columns = ['singer_id']

In [32]:
idol_list['singer_id'] = 0

In [33]:
idol_list.update(id)

In [34]:
idol_list

,artist,artist_eng,member_num,leader_name,gender,agency,singer_id
0,서태지와 아이들,SeoTaiji And Boys,3,서태지,남,요요 기획,100161
1,더 블루,The Blue,2,손지창,남,SM 엔터테인먼트,180144
2,잼,Zam,5,조진수,혼,DSP 미디어,101781
3,에스오에스,S.O.S,4,NaN,여,삼포니 레코드,
4,솔리드,Solid,3,정재윤,남,비앤비,100037
5,듀스,DEUX,2,이현도,남,예당 음향 (예당 컴퍼니),100977
6,노이즈,Noise,4,홍종구,남,라인 기획,100025


In [35]:
idol_list=idol_list.drop(idol_list.loc[idol_list['singer_id']==''].index)

In [36]:
singer_id=idol_list['singer_id']

In [37]:
title_ary=[]
album_ary=[]
date_ary=[]
genre_ary=[]
lyrics_ary=[]
lyricist_ary=[]
composer_ary=[]
singer_ary=[]
gender_ary=[]

In [38]:
singer_id

0    100161
1    180144
2    101781
4    100037
5    100977
6    100025
Name: singer_id, dtype: object

In [39]:
def search(id,n):   # 가수id를 받아 (제목,가수,가사,앨범,발매일,장르,작사,작곡)  n개 만큼탐색  
    #id에 맞는 가수의 곡 페이지(인기순)으로 이동
    driver.get('https://www.melon.com/artist/song.htm?artistId='+str(id)+'#params%5BlistType%5D=A&params%5BorderBy%5D=POPULAR_SONG_LIST&params%5BartistId%5D='+str(id)+'&po=pageObj&startIndex=1')
    driver.implicitly_wait(1)
    for i in range(1,n+1):

        driver.execute_script('window.scrollTo(0,200)')
        
        # i번째 곡정보 클릭
        driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+str(i)+']/td[3]/div/div/a[1]').click()
        
        #제목 가수 가사 앨범 발매일 장르 작사 작곡 정보 리스트에 저장
        title=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]')
        title_ary.append(title.text)
    
        singer=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]')
        singer_ary.append(singer.text)
    
        album=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a')
        album_ary.append(album.text)
    
        date=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]')
        date_ary.append(date.text)
    
        genre=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]')
        genre_ary.append(genre.text)
        
        # 가사,작사,작곡이 없는경우가 있어 예외처리
        try:
            lyricist=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[1]/div[2]/div[1]/a')
            lyricist_ary.append(lyricist.text)
        except:
            lyricist_ary.append('NULL')
        
        try:
            composer=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[2]/div[2]/div[1]/a')
            composer_ary.append(composer.text)
        except:
            composer_ary.append('NULL')
        
        try:
            lyrics=driver.find_element_by_xpath('//*[@id="d_video_summary"]')
            lyrics_ary.append(lyrics.text.replace('\n',' '))
        except:
            lyrics_ary.append('NULL')
    
            
        driver.back() #뒤로가기 ->다시 곡 목록으로 이동
    
    
    
    
    
    
    
    
    

In [40]:
for i in singer_id:
   search(i,15)



In [41]:

df=pd.DataFrame({"년도":date_ary,"artist":singer_ary,"제목":title_ary,"앨범":album_ary,"장르":genre_ary,
             "작사가":composer_ary,"작곡가":lyricist_ary,"가사":lyrics_ary})

In [42]:
print(df)

            년도    artist                    제목                    앨범   장르  \
0   1992.03.23  서태지와 아이들                 난 알아요            1집 - 난 알아요   댄스   
1   1995.10.05  서태지와 아이들        Come Back Home              4집 - 컴백홈   댄스   
2   1996.02.10  서태지와 아이들                   너에게    Goodbye Best Album  발라드   
3   1996.02.10  서태지와 아이들                   하여가    Goodbye Best Album   댄스   
4   1993.06.06  서태지와 아이들              하여가(何如歌)              2집 - 하여가   댄스   
..         ...       ...                   ...                   ...  ...   
85     1992.11       노이즈                    변명           Sound Shock   댄스   
86     1994.03       노이즈        우리가 빛이 될 수 있다면                Noise2  발라드   
87     1995.04       노이즈            소유할 수 없는 너  Noise 3rd Revolution   댄스   
88     1995.11       노이즈                    이젠    Breakin‘ The Noise   댄스   
89     1994.03       노이즈  너에게 원한 건 (Multi Mix)                Noise2   댄스   

             작사가  작곡가                                                 가사  


In [43]:
df_new=pd.merge(df,idol_list, on="artist", how="outer")

In [44]:
df_new_1=df_new.drop(['artist_eng','member_num','leader_name','singer_id'],axis=1)

In [46]:
df_new_1.rename(columns={"artist":"가수"}, inplace=True)
df_new_1.rename(columns={"gender":"성별"}, inplace=True)
df_new_1.rename(columns={"agency":"소속사"}, inplace=True)

In [50]:
df_new_1

,년도,가수,제목,앨범,장르,작사가,작곡가,가사,성별,소속사
0,1992.03.23,서태지와 아이들,난 알아요,1집 - 난 알아요,댄스,서태지,서태지,난 알아요 이밤이 흐르고 흐르면 누군가가 나를 떠나 버려야 한다는 그 사실을 그 이...,남,요요 기획
1,1995.10.05,서태지와 아이들,Come Back Home,4집 - 컴백홈,댄스,서태지,서태지,난 지금 무엇을 찾으려고 애를 쓰는 걸까 난 지금 어디로 쉬지않고 흘러가는가 난 내...,남,요요 기획
2,1996.02.10,서태지와 아이들,너에게,Goodbye Best Album,발라드,WILLIAM BYUN,서태지,니가 아무리 지금 날 좋아한다고 해도 그건 지금뿐일지도 몰라 왜냐하면 어 그건 말야...,남,요요 기획
3,1996.02.10,서태지와 아이들,하여가,Goodbye Best Album,댄스,서태지,서태지,너에게 모든걸 뺏겨버렸던 마음이 다시 내게 돌아오는 걸 느꼈지 너는 언제까지나 나만...,남,요요 기획
4,1993.06.06,서태지와 아이들,하여가(何如歌),2집 - 하여가,댄스,서태지,서태지,너에게 모든 걸 뺏겨 버렸던 마음이 다시 내게 돌아 오는 걸 느꼈지 너는 언제까지나...,남,요요 기획
...,...,...,...,...,...,...,...,...,...,...
85,1992.11,노이즈,변명,Sound Shock,댄스,김창환,김창환,이젠 너의 변명처럼 너는 나를 떠나겠지만 아무 말도 필요없잖아 이미 알고 있던 이별...,남,라인 기획
86,1994.03,노이즈,우리가 빛이 될 수 있다면,Noise2,발라드,천성일,천성일,우리 이젠 다시 일어서야지 이곳은 아직도 밝은 빛이 우릴 비추잖니 너 슬퍼 하지마 ...,남,라인 기획
87,1995.04,노이즈,소유할 수 없는 너,Noise 3rd Revolution,댄스,천성일,김창환,너 때문이란 말은 안하겠어 내가 먼저 싫증이 나버렸던거야 너로 인해 생긴 아주 많은...,남,라인 기획
88,1995.11,노이즈,이젠,Breakin‘ The Noise,댄스,김창환,김창환,이젠 이젠 너의 맘이 변한 이유를 너의 너의 이미 내가 지워지고 없었어 이젠 이젠 ...,남,라인 기획


In [51]:
df_new_1=df_new_1[["년도","가수","제목","성별","장르","작사가","작곡가","소속사","가사"]]

In [52]:
df_new_1

,년도,가수,제목,성별,장르,작사가,작곡가,소속사,가사
0,1992.03.23,서태지와 아이들,난 알아요,남,댄스,서태지,서태지,요요 기획,난 알아요 이밤이 흐르고 흐르면 누군가가 나를 떠나 버려야 한다는 그 사실을 그 이...
1,1995.10.05,서태지와 아이들,Come Back Home,남,댄스,서태지,서태지,요요 기획,난 지금 무엇을 찾으려고 애를 쓰는 걸까 난 지금 어디로 쉬지않고 흘러가는가 난 내...
2,1996.02.10,서태지와 아이들,너에게,남,발라드,WILLIAM BYUN,서태지,요요 기획,니가 아무리 지금 날 좋아한다고 해도 그건 지금뿐일지도 몰라 왜냐하면 어 그건 말야...
3,1996.02.10,서태지와 아이들,하여가,남,댄스,서태지,서태지,요요 기획,너에게 모든걸 뺏겨버렸던 마음이 다시 내게 돌아오는 걸 느꼈지 너는 언제까지나 나만...
4,1993.06.06,서태지와 아이들,하여가(何如歌),남,댄스,서태지,서태지,요요 기획,너에게 모든 걸 뺏겨 버렸던 마음이 다시 내게 돌아 오는 걸 느꼈지 너는 언제까지나...
...,...,...,...,...,...,...,...,...,...
85,1992.11,노이즈,변명,남,댄스,김창환,김창환,라인 기획,이젠 너의 변명처럼 너는 나를 떠나겠지만 아무 말도 필요없잖아 이미 알고 있던 이별...
86,1994.03,노이즈,우리가 빛이 될 수 있다면,남,발라드,천성일,천성일,라인 기획,우리 이젠 다시 일어서야지 이곳은 아직도 밝은 빛이 우릴 비추잖니 너 슬퍼 하지마 ...
87,1995.04,노이즈,소유할 수 없는 너,남,댄스,천성일,김창환,라인 기획,너 때문이란 말은 안하겠어 내가 먼저 싫증이 나버렸던거야 너로 인해 생긴 아주 많은...
88,1995.11,노이즈,이젠,남,댄스,김창환,김창환,라인 기획,이젠 이젠 너의 맘이 변한 이유를 너의 너의 이미 내가 지워지고 없었어 이젠 이젠 ...


In [53]:
df_new_1.to_csv('k-pop_92-93.tsv',index=False,header=None,sep='\t') #tsv파일로 저장